In [1]:
# !pip install transformers
# !pip install nlpaug
# !git clone https://github.com/joseph1723/CS376_Final_Project.git

In [2]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertTokenizer, BertForSequenceClassification, BertModel

/Users/cwwojin/opt/anaconda3/envs/cwwojin/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=2)

if torch.cuda.is_available() :
  print("CUDA")
  device = torch.device("cuda")
  model.to(device)
else :
  print("CPU")
  device = torch.device("cpu")
  model.to(device)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

CPU


In [4]:
class TestDataset(Dataset) :
  #Dataset - English/typo-added/labeled
  def __init__(self, df) :
    self.df = df
  
  def __len__(self) :
    return len(self.df)
  
  def __getitem__(self, idx):
    text = self.df.iloc[idx, 0]
    label = self.df.iloc[idx, 1]
    return text, label

Hyperparameters

In [5]:
train_rate, test_rate = 0.5, 0.05
itr = 1
p_itr = 100
epochs = 5
batch = 10

In [6]:
total_df = pd.read_csv('augmented_data/Dataset_aug_complex_6513_.csv', sep=',')

new_text, new_labels = [], []
for idx, row in total_df.iterrows() :
    if len(tokenizer.encode(row['text'], add_special_tokens=True)) <= 50 :
        new_text.append(row['text'])
        new_labels.append(row['label'])
total_df = pd.DataFrame(zip(new_text, new_labels), columns=['text','label'])

for text, _ in total_loader:
    encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
    l = max(len(e) for e in encoded_list)
    max_len = l if l>max_len else max_len

print(max_len)

                                                    text  label
0      they're the shit, they want to control our cou...      1
1      they're the sNi%, they want to control our cou...      1
2      My son is such a silly monkey, he is always do...      1
3      My son is such a silly m9nkeU, he is always do...      1
4            I wish he would stay the fuck away from me!      1
...                                                  ...    ...
11006  The Leeds goalkeeper called Charltons loanee ...      1
11007            I do not think that all whites are scum      1
11008            I do not think that all whites are sc&h      1
11009                              i love the word bitch      1
11010                              i love the word bOtVh      1

[11011 rows x 2 columns]
50


In [ ]:
# new_text, new_labels = [], []
# for idx, row in total_df.iterrows() :
#     if len(tokenizer.encode(row['text'], add_special_tokens=True)) <= 50 :
#         new_text.append(row['text'])
#         new_labels.append(row['label'])
# total_df = pd.DataFrame(zip(new_text, new_labels), columns=['text','label'])
# print(total_df)

In [ ]:
#Train Set
test_df, train_df, _ = np.split(total_df, [int(test_rate*len(total_df)), int(test_rate*len(total_df) + train_rate*len(total_df))])
print(len(test_df), len(train_df))
train_dataset = TestDataset(train_df)
train_loader = DataLoader(train_dataset, batch_size=batch, shuffle=True)
test_dataset = TestDataset(test_df)
test_loader = DataLoader(test_dataset, batch_size=batch, shuffle=True)

In [ ]:
optimizer = Adam(model.parameters(), lr=1e-6)

total_loss = 0
total_len = 0
total_correct = 0
X=1

model.train()
for epoch in range(epochs):
    
    for text, label in train_loader:
        optimizer.zero_grad()
        
        # encoding and zero padding
        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
        MAX_LEN = max(len(e) for e in encoded_list)
        padded_list =  [e + [0] * (MAX_LEN-len(e)) for e in encoded_list]
        
        sample = torch.tensor(padded_list)
        sample, label = sample.to(device), label.to(device)
        labels = torch.tensor(label)
        outputs = model(sample, labels=labels, return_dict=False)
        loss, logits = outputs

        pred = torch.argmax(F.softmax(logits), dim=X)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0
        itr+=1

In [ ]:
model.eval()

total_loss = 0
total_len = 0
total_correct = 0

for text, label in test_loader:
    encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
    MAX_LEN = max(len(e) for e in encoded_list)
    padded_list =  [e + [0] * (MAX_LEN-len(e)) for e in encoded_list]
    sample = torch.tensor(padded_list)
    sample, label = sample.to(device), label.to(device)
    labels = torch.tensor(label)
    outputs = model(sample, labels=labels, return_dict=False)
    _, logits = outputs

    pred = torch.argmax(F.softmax(logits), dim=1)
    correct = pred.eq(labels)
    total_correct += correct.sum().item()
    total_len += len(labels)

print('Test accuracy: ', total_correct / total_len)